In [1]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import os


os.makedirs('../models', exist_ok=True)


df = pd.read_csv('../data/processed/cleaned_disaster_data.csv')


def create_features(df):

    location_encoder = LabelEncoder()
    df['location_encoded'] = location_encoder.fit_transform(df['location'])
    

    disaster_encoder = LabelEncoder()
    df['disaster_encoded'] = disaster_encoder.fit_transform(df['disaster_type'])
    

    df['season'] = df['month'].apply(lambda x: 
        'Spring' if x in [3,4,5] else
        'Summer' if x in [6,7,8] else
        'Fall' if x in [9,10,11] else 'Winter')
    

    season_encoder = LabelEncoder()
    df['season_encoded'] = season_encoder.fit_transform(df['season'])
    

    location_risk = df.groupby('location').size() / len(df)
    df['location_risk'] = df['location'].map(location_risk)
    

    disaster_freq = df.groupby('disaster_type').size() / len(df)
    df['disaster_freq'] = df['disaster_type'].map(disaster_freq)
    
    return df, location_encoder, disaster_encoder, season_encoder


def train_predictive_model(X, y):

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    return model, X_test, y_test

def evaluate_model(model, X_test, y_test):

    y_pred = model.predict(X_test)
    

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    return accuracy, report, cm


df, location_encoder, disaster_encoder, season_encoder = create_features(df)


features = ['year', 'month', 'day', 'day_of_week', 'location_encoded', 
            'season_encoded', 'location_risk', 'disaster_freq']
X = df[features]
y = df['disaster_encoded']


model, X_test, y_test = train_predictive_model(X, y)


accuracy, report, cm = evaluate_model(model, X_test, y_test)
print(f"Model Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


joblib.dump(model, '../models/disaster_predictor.pkl')
joblib.dump(location_encoder, '../models/location_encoder.pkl')
joblib.dump(disaster_encoder, '../models/disaster_encoder.pkl')
joblib.dump(season_encoder, '../models/season_encoder.pkl')

print("Model and encoders saved successfully!")

c:\Users\CALEB ASSAN\Documents\Predictive\env\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\CALEB ASSAN\Documents\Predictive\env\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\CALEB ASSAN\Documents\Predictive\env\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c

Model Accuracy: 0.93
Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           8       1.00      1.00      1.00        35
           9       1.00      1.00      1.00        36
          11       0.00      0.00      0.00         1
          15       1.00      0.33      0.50         3
          16       0.00      0.00      0.00         1
          18       0.93      1.00      0.96        27
          19       0.00      0.00      0.00         2
          20       0.79      1.00      0.88        19

    accuracy                           0.93       127
   macro avg       0.47      0.43      0.43       127
weighted avg       0.90      0.93      0.91       127

Model and encoders saved successfully!


In [4]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import os


os.makedirs('../models', exist_ok=True)


df = pd.read_csv('../data/processed/cleaned_disaster_data.csv')


def create_features(df):

    location_encoder = LabelEncoder()
    df['location_encoded'] = location_encoder.fit_transform(df['location'])
    

    disaster_encoder = LabelEncoder()
    df['disaster_encoded'] = disaster_encoder.fit_transform(df['disaster_type'])
    

    df['season'] = df['month'].apply(lambda x: 
        'Spring' if x in [3,4,5] else
        'Summer' if x in [6,7,8] else
        'Fall' if x in [9,10,11] else 'Winter')
    

    all_seasons = ['Spring', 'Summer', 'Fall', 'Winter']
    existing_seasons = df['season'].unique()
    
    missing_seasons = [s for s in all_seasons if s not in existing_seasons]
    if missing_seasons:
        print(f"Adding dummy rows for missing seasons: {missing_seasons}")
        for season in missing_seasons:

            dummy_row = df.iloc[0].copy()
            dummy_row['season'] = season

            if season == 'Spring':
                dummy_row['month'] = 4
            elif season == 'Summer':
                dummy_row['month'] = 7
            elif season == 'Fall':
                dummy_row['month'] = 10
            else:  
                dummy_row['month'] = 1
            
    
            df = pd.concat([df, dummy_row.to_frame().T], ignore_index=True)
    

    season_encoder = LabelEncoder()
    df['season_encoded'] = season_encoder.fit_transform(df['season'])
    

    location_risk = df.groupby('location').size() / len(df)
    df['location_risk'] = df['location'].map(location_risk)
    

    disaster_freq = df.groupby('disaster_type').size() / len(df)
    df['disaster_freq'] = df['disaster_type'].map(disaster_freq)
    
    return df, location_encoder, disaster_encoder, season_encoder


def train_predictive_model(X, y):

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    return model, X_test, y_test

def evaluate_model(model, X_test, y_test):

    y_pred = model.predict(X_test)
    

    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    return accuracy, report, cm


df, location_encoder, disaster_encoder, season_encoder = create_features(df)


print("Seasons in data:", season_encoder.classes_)
print("Disaster types in data:", disaster_encoder.classes_)


features = ['year', 'month', 'day', 'day_of_week', 'location_encoded', 
            'season_encoded', 'location_risk', 'disaster_freq']
X = df[features]
y = df['disaster_encoded']


model, X_test, y_test = train_predictive_model(X, y)


print("Model classes:", model.classes_)


if not np.array_equal(model.classes_, disaster_encoder.classes_):
    print("WARNING: Model classes do not match disaster encoder classes!")
    print("Model classes:", model.classes_)
    print("Disaster encoder classes:", disaster_encoder.classes_)
    
    disaster_encoder.classes_ = model.classes_
    print("Updated disaster encoder classes to match model classes")


accuracy, report, cm = evaluate_model(model, X_test, y_test)
print(f"Model Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


joblib.dump(model, '../models/disaster_predictor.pkl')
joblib.dump(location_encoder, '../models/location_encoder.pkl')
joblib.dump(disaster_encoder, '../models/disaster_encoder.pkl')
joblib.dump(season_encoder, '../models/season_encoder.pkl')


joblib.dump(model.classes_, '../models/model_classes.pkl')


disaster_mapping = {i: disaster_type for i, disaster_type in enumerate(model.classes_)}
joblib.dump(disaster_mapping, '../models/disaster_mapping.pkl')

print("Model and encoders saved successfully!")
print("Disaster mapping:", disaster_mapping)

Seasons in data: ['Fall' 'Spring' 'Summer' 'Winter']
Disaster types in data: ['ACCIDENT' 'COLLAPSE' 'COMERCIAL FIRE' 'DISASTER_TYPE' 'DROWN' 'DROWNING'
 'EPIDEMIC' 'EXPLOSION' 'FIRE' 'FLOOD' 'INDUSTRIAL FIRE' 'LANDSLIDE'
 'LIGHTNING' 'MAN MADE (BUILDING COLLAPSE)' 'MAN MADE(BUILDING COLLAPSE)'
 'MAN_MADE' 'PEST INFESTATION' 'PEST_INFESTATION' 'RAIN_STORM'
 'TIDAL_WAVE' 'WIND_STORM']
Model classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20]
Model classes: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20]
Disaster encoder classes: ['ACCIDENT' 'COLLAPSE' 'COMERCIAL FIRE' 'DISASTER_TYPE' 'DROWN' 'DROWNING'
 'EPIDEMIC' 'EXPLOSION' 'FIRE' 'FLOOD' 'INDUSTRIAL FIRE' 'LANDSLIDE'
 'LIGHTNING' 'MAN MADE (BUILDING COLLAPSE)' 'MAN MADE(BUILDING COLLAPSE)'
 'MAN_MADE' 'PEST INFESTATION' 'PEST_INFESTATION' 'RAIN_STORM'
 'TIDAL_WAVE' 'WIND_STORM']
Updated disaster encoder classes to match model classes
Model Accuracy: 0.93
Classification Report:
              precision  

c:\Users\CALEB ASSAN\Documents\Predictive\env\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\CALEB ASSAN\Documents\Predictive\env\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\CALEB ASSAN\Documents\Predictive\env\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.c